In [1]:
import numpy as np
from numba import cuda

# Check if a GPU is available
if cuda.is_available():
    print("GPU is available.")
    
    # Get the name of the GPU
    gpu_name = cuda.get_current_device().name
    print(f"Using GPU: {gpu_name}")
    
else:
    print("GPU is not available. Using CPU.")

# Your CUDA code here, if you want to run specific CUDA operations



GPU is not available. Using CPU.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input
from numba import jit, cuda

2023-10-06 18:12:23.833737: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 18:12:23.874438: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-06 18:12:23.874472: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-06 18:12:23.874499: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-06 18:12:23.881510: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 18:12:23.881956: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
df = pd.read_csv('data/preprocessed_data.csv', encoding='utf-8')
df = df.head(1000)

In [5]:
hindi_sentences = df['Hindi']
telugu_sentences = df['Telugu']
hindi_train, hindi_val, telugu_train, telugu_val = train_test_split(hindi_sentences, telugu_sentences, test_size=0.2, random_state=42)

In [6]:
hindi_tokenizer = Tokenizer(oov_token="<OOV>")
hindi_tokenizer.fit_on_texts(hindi_sentences)
hindi_vocab_size = len(hindi_tokenizer.word_index) + 1

In [7]:
telugu_tokenizer = Tokenizer(oov_token="<OOV>")
telugu_tokenizer.fit_on_texts(telugu_sentences)
telugu_vocab_size = len(telugu_tokenizer.word_index) + 1

In [8]:
max_sequence_length = max(len(seq) for seq in hindi_train)
hindi_train_sequences = pad_sequences(hindi_tokenizer.texts_to_sequences(hindi_train), maxlen=max_sequence_length, padding="post")
telugu_train_sequences = pad_sequences(telugu_tokenizer.texts_to_sequences(telugu_train), maxlen=max_sequence_length, padding="post")


In [9]:
def gpu_evaluate_model(model, hindi_sequences, telugu_sequences, target_test_data):
    evaluation = model.evaluate([hindi_sequences, telugu_sequences], target_test_data)
    return evaluation

In [10]:
def create_gpu_model(hindi_vocab_size, telugu_vocab_size, max_sequence_length):
    encoder_inputs = Input(shape=(max_sequence_length,))
    encoder_embedding = Embedding(hindi_vocab_size, 256, input_length=max_sequence_length)(encoder_inputs)
    encoder_lstm = LSTM(256, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(max_sequence_length,))
    decoder_embedding = Embedding(telugu_vocab_size, 256, input_length=max_sequence_length)(decoder_inputs)
    decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(telugu_vocab_size, activation="softmax")
    output = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], output)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [11]:
gpu_model = create_gpu_model(hindi_vocab_size, telugu_vocab_size, max_sequence_length)
epochs = 5

2023-10-06 18:12:55.868390: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-06 18:12:55.869196: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
import tensorflow as tf
target_train_data = tf.keras.utils.to_categorical(telugu_train_sequences, num_classes=telugu_vocab_size, dtype='float32')

In [13]:
gpu_model.fit([hindi_train_sequences, telugu_train_sequences], target_train_data, epochs=epochs, batch_size=64, validation_split=0.2)

Epoch 1/5


2023-10-06 18:13:08.833714: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5957529600 exceeds 10% of free system memory.
2023-10-06 18:13:13.909869: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 595752960 exceeds 10% of free system memory.
2023-10-06 18:13:13.911056: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 595752960 exceeds 10% of free system memory.
2023-10-06 18:13:13.911361: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 595752960 exceeds 10% of free system memory.
2023-10-06 18:13:13.912013: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 595752960 exceeds 10% of free system memory.


10/10 [==============================] - 59s 6s/step - loss: 7.0471 - accuracy: 0.8757 - val_loss: 3.9726 - val_accuracy: 0.9725
Epoch 2/5
10/10 [==============================] - 53s 6s/step - loss: 1.6969 - accuracy: 0.9728 - val_loss: 0.3434 - val_accuracy: 0.9725
Epoch 3/5
10/10 [==============================] - 52s 5s/step - loss: 0.2813 - accuracy: 0.9728 - val_loss: 0.2596 - val_accuracy: 0.9725
Epoch 4/5
10/10 [==============================] - 56s 6s/step - loss: 0.2532 - accuracy: 0.9728 - val_loss: 0.2552 - val_accuracy: 0.9725
Epoch 5/5
10/10 [==============================] - 55s 6s/step - loss: 0.2481 - accuracy: 0.9728 - val_loss: 0.2546 - val_accuracy: 0.9725


In [14]:
hindi_test_sequences = pad_sequences(hindi_tokenizer.texts_to_sequences(hindi_val), maxlen=max_sequence_length, padding="post")
telugu_test_sequences = pad_sequences(telugu_tokenizer.texts_to_sequences(telugu_val), maxlen=max_sequence_length, padding="post")

target_test_data = tf.keras.utils.to_categorical(telugu_test_sequences, num_classes=telugu_vocab_size, dtype='float32')

evaluation = gpu_evaluate_model(gpu_model, hindi_test_sequences, telugu_test_sequences, target_test_data)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

7/7 [==============================] - 6s 788ms/step - loss: 0.2516 - accuracy: 0.9731
Test Loss: 0.25158432126045227
Test Accuracy: 0.9730697870254517


In [15]:
import pickle

gpu_model.save('models/gpu_translation_model.keras')

with open('tokenizers/gpu_hindi_tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(hindi_tokenizer, tokenizer_file)

with open('tokenizers/gpu_telugu_tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(telugu_tokenizer, tokenizer_file)